In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException,TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import getData as GD
from datetime import date
import glob
import os
import traceback

# Df BASE

In [2]:
############Configuro la extension para on crear duplicados#############
chrome_options = Options()
chrome_options.add_extension('Extensione/1.0.0_0.crx')
driver = webdriver.Chrome(options=chrome_options)
############ Me Logeo en el Mapa Movil ###################
driver.get("https://172.19.112.62/central_login/index.php?action=remoteAuthenticacion&service=http://172.16.102.103/Mapa_Evolucion_Clientes/&apiKey=a56ggrL4qyfr5aAcTJf5leUrkXgfsdfs4")
button = driver.find_element(By.ID, "details-button")
button.click()
acces = driver.find_element(By.ID, "proceed-link")
acces.click()
GD.login(driver,'username','password')
###############LOGIN########################################################
driver.get("http://172.16.102.103/Mapa_Evolucion_Clientes/map/show/6500000")
assert "Clientes Moviles | Claro Peru" in driver.title, 'Pagina Caida' #reviso el estado de la pagina
time.sleep(4.5) ############ espero a que cargue Cambiar por espera explicita 
############### Set Granulidad"#########################
Granulidad = driver.find_element(By.XPATH, ('/html/body/div[1]/div/div[2]/div/nav/div/div[3]/div/div[1]/li/a/span'))
Granulidad.click()
Granulidad_botton = driver.find_element(By.XPATH, ('//*[@id="map_distrito"]')) 
Granulidad_botton.click()
fecha_actual = date.today()
# Convertir la fecha en formato de cadena de caracteres (string)
fecha_str = fecha_actual.strftime('%d-%m-%Y')
descarga = driver.find_element(By.XPATH, ('//*[@id="table-search"]/div[9]/button')) ######### find the botton of download
descarga.click() 
time.sleep(3)


# Proceso el DF

## Filtro por el norte y corrigo las ñ

In [3]:
Correguir = {
    'NEPEÃ‘A': 'NEPEÑA',
    'SAÃ‘A': 'SAÑA',
    'FERREÃ‘AFE': 'FERREÑAFE',
    'CAÃ‘ARIS': 'CAÑARIS',
    'CHANCAYBAÃ‘OS': 'CHANCAYBAÑOS',
    'ENCAÃ‘ADA': 'ENCAÑADA',
    'LOS BAÃ‘OS DEL INCA' : 'LOS BAÑOS DEL INCA',
    'PARIÃ‘AS' : 'PARIÑAS' 
}


In [4]:
PathofE = 'C:/Users/C26764/Downloads/file_name.csv' #Cargo el archivo que acabo de descargar 
Norte = ['AMAZONAS', 'ANCASH', 'CAJAMARCA', 'LA LIBERTAD', 'LAMBAYEQUE', 'TUMBES','PIURA']
CPX = pd.read_csv(PathofE)
CPX = CPX.rename(columns={'Departament' : 'Departamento', #renombro
                    'Provincie': 'Provincia', 
                    'Disrito' : 'Distrito'})
##################### Filtroy doy formato###################
CPX_N = CPX[CPX['Departamento'].isin(Norte)]
CPX_N.loc[:, 'Departamento'] = CPX_N['Departamento'].str.title()
################# Corrigo las ñs#######################################
CPX_N.loc[:,'Distrito'] = CPX_N['Distrito'].replace(Correguir)
CPX_N.loc[:,'Provincia'] = CPX_N['Provincia'].replace(Correguir)

# Scrap

In [5]:
Departamentos = CPX_N['Departamento'].unique() 
inicio = time.time()
Data_2_extrac = ["Trafico Datos 2G / 3G / 4G *","Velocidad Bajada Osiptel (Mbps)","Mayor Competidor","Mejor Red",
                 "Usuarios Uso de Red","Departamento / Provincia / Distrito","Pérdida de paquetes Osiptel",
                 "Customer Exp. Index (CEI)","Tiempo de Cobertura Osiptel "]
# Itera sobre cada departamento único en CPX_N
Extractor = GD.Extract_Data(Data_2_extrac) #Defino una instancia de clase

for Departamento in Departamentos:
     # Selecciona el departamento en la página web
    GD.select_button(driver,'reg', lugar = Departamento)
    print(Departamento, ' Start')
    
    # Itera sobre cada provincia única en el departamento actual
    for Provincia in CPX_N[CPX_N['Departamento'] == Departamento]['Provincia'].unique():
        print('Start: ', Provincia)
        try:
            GD.select_button(driver,'prov',lugar=Provincia)
        except (TimeoutException):
            print('Salto la Excepcion en: ',Departamento)
            out = driver.find_element(By.XPATH,('/html/body/div[2]'))
            out.click()
         # Espera hasta que la lista de distritos esté presente
        wait = WebDriverWait(driver, 8)
        Lista_de_Distritos = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dist-ul"]'))) #Localizo la lista de dist
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dist-ul"]/li[2]'))) # Espera que esten la lista de distritos        
        Distritos = Lista_de_Distritos.find_elements(By.XPATH, 'li') # Encuentra todos los elementos de distrito en la lista de distritos    
        # Imprime el número de elementos de distritos en la lista desplegable y los que se usarán 
        Dist_of_df = CPX_N[(CPX_N['Departamento'] == Departamento) & (CPX_N['Provincia'] == Provincia)]['Distrito'].tolist() 
        #print(f'Número de elementos en {Provincia}: {len(Distritos)} y solo usare {len(Dist_of_df)}' ) 
        
        # Itera sobre cada distrito en la lista de distritos
        print(Dist_of_df)
        
        for Distrito in Distritos:
            try:
                #print(Distrito.find_element(By.CSS_SELECTOR, 'a').get_attribute('value'))
                GD.DataofDist(Extractor,Distrito, Dist_of_df, driver)    
            except (ElementClickInterceptedException, TimeoutException) as e:
                out = wait.until(EC.element_to_be_clickable((By.XPATH,('/html/body/div[2]'))))
                print(f"Se produjo una excepción: {type(e).__name__}")
                print(str(e))
                print(traceback.format_exc())
                #out.click()
                print('Salto la Excepcion en: ',Dist_element_name) 
                
                 
Data_ofDist = Extractor.to_df()
final = time.time()
duracion = final - inicio
print("End of Scrap","Tardo: ",duracion,' Segundos en ejecutarse y se extrajeron: ',len(Data_ofDist), 'filas')
driver.close()


Amazonas  Start
Start:  BAGUA
['IMAZA', 'ARAMANGO', 'BAGUA', 'LA PECA']
Click en:  BAGUA
Luego de:  0 Intentos, coincidencia en:  BAGUA
Panel listo,itero para extraer la data
Se extrajo la data de:  AMAZONAS / BAGUA / BAGUA
Click en:  ARAMANGO
Luego de:  0 Intentos, coincidencia en:  ARAMANGO
Panel listo,itero para extraer la data
Se extrajo la data de:  AMAZONAS / BAGUA / ARAMANGO
Click en:  IMAZA
Luego de:  0 Intentos, coincidencia en:  IMAZA
Panel listo,itero para extraer la data
Se extrajo la data de:  AMAZONAS / BAGUA / IMAZA
Click en:  LA PECA
Luego de:  0 Intentos, coincidencia en:  LA PECA
Panel listo,itero para extraer la data
Se extrajo la data de:  AMAZONAS / BAGUA / LA PECA
Start:  CONDORCANQUI
['NIEVA']
Click en:  NIEVA
Luego de:  0 Intentos, coincidencia en:  NIEVA
Panel listo,itero para extraer la data
Se extrajo la data de:  AMAZONAS / CONDORCANQUI / NIEVA
Start:  UTCUBAMBA
['CAJARURO', 'BAGUA GRANDE', 'JAMALCA', 'LONYA GRANDE', 'EL MILAGRO']
Click en:  BAGUA GRANDE
Lue

ElementClickInterceptedException: Message: element click intercepted: Element <div style="z-index: 3; position: absolute; height: 100%; width: 100%; padding: 0px; border-width: 0px; margin: 0px; left: 0px; top: 0px; touch-action: pan-x pan-y;">...</div> is not clickable at point (794, 371). Other element would receive the click: <div class="bootbox modal fade in" tabindex="-1" role="dialog" style="display: block; padding-right: 17px;">...</div>
  (Session info: chrome=121.0.6167.139)
Stacktrace:
	GetHandleVerifier [0x00007FF7CF235E42+3538674]
	(No symbol) [0x00007FF7CEE54C02]
	(No symbol) [0x00007FF7CED05AEB]
	(No symbol) [0x00007FF7CED533E4]
	(No symbol) [0x00007FF7CED51122]
	(No symbol) [0x00007FF7CED4E884]
	(No symbol) [0x00007FF7CED4D5A9]
	(No symbol) [0x00007FF7CED41950]
	(No symbol) [0x00007FF7CED6F0AA]
	(No symbol) [0x00007FF7CED4124A]
	(No symbol) [0x00007FF7CED6F2C0]
	(No symbol) [0x00007FF7CED8BDE3]
	(No symbol) [0x00007FF7CED6EE53]
	(No symbol) [0x00007FF7CED3F514]
	(No symb

In [ ]:
Data_ofDist.to_csv('Temporaldf.csv',index=False) #Para cuando quiero procesar en otro timepo del scrap


In [6]:
Data_ofDist

,Trafico Datos 2G / 3G / 4G *,Velocidad Bajada Osiptel (Mbps)\nClaro / Movistar / Entel / Bitel *,Mayor Competidor,Mejor Red,Usuarios Uso De Red\nSolo Encendidos / Solo Voz / Datos *,Departamento / Provincia / Distrito,Pérdida De Paquetes Osiptel\nClaro / Movistar / Entel / Bitel *,Customer Exp. Index (CEI)\nBueno/Regular/Malo *,Tiempo De Cobertura Osiptel\nClaro / Movistar / Entel / Bitel *
0,0.00% / 3.71% / 96.29%,13.56 / 10.79 / 9.20 / 8.59,ENTEL,CLARO,2.65% / 8.35% / 89.00%,AMAZONAS / BAGUA / BAGUA,0.30 % / 0.66 % / 1.01 % / 1.58 %,91.02% / 5.62% / 3.36%,96.58 % / 93.81 % / 89.71 % / 95.38 %
1,0.00% / 11.18% / 88.81%,13.82 / 6.58 / 0.00 / 7.90,ENTEL,CLARO,4.45% / 17.96% / 77.59%,AMAZONAS / BAGUA / ARAMANGO,0.78 % / 0.00 % / 0.00 % / 4.85 %,85.27% / 8.91% / 5.81%,63.16 % / 10.15 % / 12.96 % / 95.32 %
2,0.02% / 5.10% / 94.89%,10.67 / 0.00 / 0.00 / 9.26,MOVISTAR,CLARO,3.33% / 15.28% / 81.40%,AMAZONAS / BAGUA / IMAZA,1.72 % / 0.00 % / 0.00 % / 3.82 %,59.72% / 18.47% / 21.81%,71.79 % / 0.52 % / 1.48 % / 86.39 %
3,0.02% / 5.10% / 94.87%,16.87 / 13.49 / 0.00 / 8.10,MOVISTAR,CLARO,3.08% / 15.24% / 81.68%,AMAZONAS / BAGUA / LA PECA,0.41 % / 0.90 % / 0.00 % / 0.34 %,86.39% / 8.70% / 4.91%,90.39 % / 95.71 % / 62.65 % / 94.37 %
4,100.00% / 0.00% / 0.00%,0.00 / 0.00 / 0.00 / 7.59,ENTEL,BITEL,10.98% / 34.23% / 54.79%,AMAZONAS / CONDORCANQUI / NIEVA,0.00 % / 0.00 % / 1.21 % / 1.74 %,30.00% / 5.00% / 65.00%,1.25 % / 18.42 % / 96.04 % / 81.57 %
...,...,...,...,...,...,...,...,...,...
354,0.00% / 3.28% / 96.71%,12.58 / 10.88 / 15.56 / 14.33,MOVISTAR,ENTEL,1.21% / 3.17% / 95.62%,TUMBES / CONTRALMIRANTE VILLAR / CANOAS DE PUN...,0.83 % / 0.48 % / 0.38 % / 0.47 %,91.56% / 5.52% / 2.92%,86.81 % / 88.30 % / 96.02 % / 97.13 %
355,0.00% / 3.55% / 96.44%,12.86 / 10.92 / 10.07 / 9.36,MOVISTAR,CLARO,2.36% / 5.16% / 92.49%,TUMBES / ZARUMILLA / ZARUMILLA,0.37 % / 0.61 % / 0.56 % / 0.61 %,92.54% / 6.15% / 1.31%,87.25 % / 89.64 % / 97.85 % / 98.02 %
356,0.00% / 2.17% / 97.82%,13.85 / 8.64 / 10.02 / 9.58,CRECIENDO,CLARO,2.41% / 3.01% / 94.59%,TUMBES / ZARUMILLA / AGUAS VERDES,0.61 % / 1.40 % / 0.86 % / 1.54 %,95.13% / 3.29% / 1.57%,97.27 % / 79.91 % / 92.97 % / 95.67 %
357,0.01% / 8.88% / 91.11%,11.70 / 0.00 / 0.00 / 0.00,BITEL,CLARO,1.71% / 8.25% / 90.04%,TUMBES / ZARUMILLA / MATAPALO,0.80 % / 0.00 % / 0.00 % / 1.10 %,88.92% / 7.98% / 3.11%,90.94 % / 0.15 % / 71.01 % / 79.21 %


# DF to csv, temporal

In [2]:
Data_ofDist.to_csv('Temporaldf.csv',index=False) #Para cuando quiero procesar en otro timepo del scrap
Data_ofDist.head(5)

NameError: name 'Data_ofDist' is not defined

# Proceso la data para darle formato y poder hace run merge con el df descargado

In [21]:
Data_ofDist = pd.read_csv('Temporaldf.csv')# Load Temporal DF


In [22]:
# Iteramos sobre las columnas del DataFrame
Data_ofDist_split = pd.DataFrame()

for col_name in Data_ofDist.columns:
    # Dividimos cada columna y expandimos el resultado en nuevas columnas
    df = Data_ofDist[col_name].str.split('/', expand=True)
    # Asignamos los nombres de las nuevas columnas
    df.columns = col_name.split('/')
    a =  col_name.split('\n')
    for column in df.columns:
        # Verificamos si el prefijo ya existe en el nombre de la columna
        if not column.startswith(a[0]) and (column not in a[0]):
            # Si no existe, añadimos el prefijo
            df.rename(columns={column: a[0] + '_' + column}, inplace=True)
    # Concatenamos el DataFrame resultante con el DataFrame unido existente
    Data_ofDist_split = pd.concat([Data_ofDist_split, df], axis=1)


In [23]:
Data_ofDist_split

,Trafico Datos 2G,3G,4G *,Velocidad Bajada Osiptel (Mbps)\nClaro,Velocidad Bajada Osiptel (Mbps)_ Movistar,Velocidad Bajada Osiptel (Mbps)_ Entel,Velocidad Bajada Osiptel (Mbps)_ Bitel *,Mayor Competidor,Mejor Red,Usuarios Uso De Red\nSolo Encendidos,...,Pérdida De Paquetes Osiptel_ Movistar,Pérdida De Paquetes Osiptel_ Entel,Pérdida De Paquetes Osiptel_ Bitel *,Customer Exp. Index (CEI)\nBueno,Customer Exp. Index (CEI)_Regular,Customer Exp. Index (CEI)_Malo *,Tiempo De Cobertura Osiptel\nClaro,Tiempo De Cobertura Osiptel_ Movistar,Tiempo De Cobertura Osiptel_ Entel,Tiempo De Cobertura Osiptel_ Bitel *
0,0.00%,3.71%,96.29%,13.56,10.79,9.20,8.59,ENTEL,CLARO,2.65%,...,0.66 %,1.01 %,1.58 %,91.02%,5.62%,3.36%,96.58 %,93.81 %,89.71 %,95.38 %
1,0.00%,11.18%,88.81%,13.82,6.58,0.00,7.90,ENTEL,CLARO,4.45%,...,0.00 %,0.00 %,4.85 %,85.27%,8.91%,5.81%,63.16 %,10.15 %,12.96 %,95.32 %
2,0.02%,5.10%,94.89%,10.67,0.00,0.00,9.26,MOVISTAR,CLARO,3.33%,...,0.00 %,0.00 %,3.82 %,59.72%,18.47%,21.81%,71.79 %,0.52 %,1.48 %,86.39 %
3,0.02%,5.10%,94.87%,16.87,13.49,0.00,8.10,MOVISTAR,CLARO,3.08%,...,0.90 %,0.00 %,0.34 %,86.39%,8.70%,4.91%,90.39 %,95.71 %,62.65 %,94.37 %
4,100.00%,0.00%,0.00%,0.00,0.00,0.00,7.59,ENTEL,BITEL,10.98%,...,0.00 %,1.21 %,1.74 %,30.00%,5.00%,65.00%,1.25 %,18.42 %,96.04 %,81.57 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,0.00%,3.28%,96.71%,12.58,10.88,15.56,14.33,MOVISTAR,ENTEL,1.21%,...,0.48 %,0.38 %,0.47 %,91.56%,5.52%,2.92%,86.81 %,88.30 %,96.02 %,97.13 %
358,0.00%,3.55%,96.44%,12.86,10.92,10.07,9.36,MOVISTAR,CLARO,2.36%,...,0.61 %,0.56 %,0.61 %,92.54%,6.15%,1.31%,87.25 %,89.64 %,97.85 %,98.02 %
359,0.00%,2.17%,97.82%,13.85,8.64,10.02,9.58,CRECIENDO,CLARO,2.41%,...,1.40 %,0.86 %,1.54 %,95.13%,3.29%,1.57%,97.27 %,79.91 %,92.97 %,95.67 %
360,0.01%,8.88%,91.11%,11.70,0.00,0.00,0.00,BITEL,CLARO,1.71%,...,0.00 %,0.00 %,1.10 %,88.92%,7.98%,3.11%,90.94 %,0.15 %,71.01 %,79.21 %


In [24]:
Data_ofDist_split.columns = Data_ofDist_split.columns.str.strip()
Data_ofDist_split.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 26 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Trafico Datos 2G                           362 non-null    object
 1   3G                                         362 non-null    object
 2   4G *                                       362 non-null    object
 3   Velocidad Bajada Osiptel (Mbps)
Claro      362 non-null    object
 4   Velocidad Bajada Osiptel (Mbps)_ Movistar  362 non-null    object
 5   Velocidad Bajada Osiptel (Mbps)_ Entel     362 non-null    object
 6   Velocidad Bajada Osiptel (Mbps)_ Bitel *   362 non-null    object
 7   Mayor Competidor                           362 non-null    object
 8   Mejor Red                                  362 non-null    object
 9   Usuarios Uso De Red
Solo Encendidos        362 non-null    object
 10  Usuarios Uso De Red_ Solo Voz         

## Preparo los df para el merge
    Corrigo la ñ

In [25]:
PathofE = 'C:/Users/C26764/Downloads/file_name.csv' #Cargo el archivo que acabo de descargar 
Norte = ['AMAZONAS', 'ANCASH', 'CAJAMARCA', 'LA LIBERTAD', 'LAMBAYEQUE', 'TUMBES','PIURA']
CPX = pd.read_csv(PathofE)
CPX = CPX.rename(columns={'Departament' : 'Departamento', #renombro
                    'Provincie': 'Provincia', 
                    'Disrito' : 'Distrito'})
##################### Filtroy doy formato###################
CPX_N = CPX[CPX['Departamento'].isin(Norte)]
CPX_N.loc[:,'Departamento'] = CPX_N['Departamento'].str.upper()
CPX_N = GD.NormDf(CPX_N)
Data_ofDist_split = GD.NormDf(Data_ofDist_split)
Data_ofDist_split = Data_ofDist_split.drop_duplicates()
############################### Merge ###################################################################
########################################################################################################
DataT_Dist = pd.merge(Data_ofDist_split,CPX_N,on=['Departamento','Provincia','Distrito'], how='right')
#DataT_Dist = DataT_Dist.drop('Speed Download FB (Compare to 1 week earlier %)', axis=1)
DataT_Dist = DataT_Dist.drop('Round Trip Time FB (Compare to 1 week earlier %)', axis=1)

DataT_Dist = DataT_Dist.iloc[:,:42]

In [26]:
# Obtén los nombres de todas las columnas excepto las últimas 16
columnas_a_modificar = DataT_Dist.columns[:-16]

# Crea un diccionario con los nombres de las columnas a modificar
nuevos_nombres = {col: col.strip().replace(" ", "").replace("*", "").replace("\n","_") for col in columnas_a_modificar}

# Usa el método rename para cambiar los nombres de las columnas
DataT_Dist = DataT_Dist.rename(columns=nuevos_nombres)


In [27]:
###### le doy formato decimal a todos los porcentajes############################
######### NUmericas##########################################
columnas = ['TraficoDatos2G','3G','4G','UsuariosUsoDeRed_SoloEncendidos','UsuariosUsoDeRed_SoloVoz','UsuariosUsoDeRed_Datos',
            'PérdidaDePaquetesOsiptel_Claro','PérdidaDePaquetesOsiptel_Movistar','PérdidaDePaquetesOsiptel_Entel','PérdidaDePaquetesOsiptel_Bitel'
            ,'CustomerExp.Index(CEI)_Bueno','CustomerExp.Index(CEI)_Regular','CustomerExp.Index(CEI)_Malo',
            'TiempoDeCoberturaOsiptel_Claro','TiempoDeCoberturaOsiptel_Movistar','TiempoDeCoberturaOsiptel_Entel','TiempoDeCoberturaOsiptel_Bitel']  # Lista de columnas
for col in columnas:
    DataT_Dist[col] = DataT_Dist[col].astype(str).str.replace('%','').astype('float') / 100.0


columnas1 = [col for col in DataT_Dist.columns if '%' in col]


for col in columnas1:
    DataT_Dist[col] = DataT_Dist[col]/100.0

############### Relleno vacios###############################
Colums = columnas + columnas1
## Relleno NaN y cambio el tipo de datos
DataT_Dist[Colums] = DataT_Dist[Colums].fillna(0.0)

###########Categoricas#######################################
DataT_Dist[['MejorRed', 'MayorCompetidor']] = DataT_Dist[['MejorRed', 'MayorCompetidor']].fillna('Sin Data')    

In [28]:
##  Este rename es solo para que la data coincida con las semanas anteriores y poder crear las series temporales
DataT_Dist = DataT_Dist.rename(columns={"TraficoDatos2G" : "TD_2G",
                            '3G' : 'TD_3G',
                            '4G' : 'TD_4G',
                            'UsuariosUsoDeRed_SoloEncendidos': 'Users_Encendidos',
                            'UsuariosUsoDeRed_SoloVoz' : 'Users_Voz',
                            'UsuariosUsoDeRed_Datos' : 'Users_Datos',
                            'VelocidadBajadaOsiptel(Mbps)_Claro' : 'DL_OSIP_Claro',
                            'VelocidadBajadaOsiptel(Mbps)_Movistar' : 'DL_OSIP_Movistar',
                            'VelocidadBajadaOsiptel(Mbps)_Entel' : 'DL_OSIP_Entel',
                            'VelocidadBajadaOsiptel(Mbps)_Bitel' : 'DL_OSIP_Bitel',
                            'PérdidaDePaquetesOsiptel_Claro' : 'PL_Claro',
                            'PérdidaDePaquetesOsiptel_Movistar' : 'PL_Movistar',
                            'PérdidaDePaquetesOsiptel_Entel' : 'PL_Entel',
                            'PérdidaDePaquetesOsiptel_Bitel' : 'PL_Bitel',
                            'CustomerExp.Index(CEI)_Bueno' :  'CEI_Bueno',
                            'CustomerExp.Index(CEI)_Regular' :  'CEI_Regular',
                            'CustomerExp.Index(CEI)_Malo' :   'CEI_Malo'})

# Exporto a csv el DF final

In [29]:
# Cuidado con canchar en la fecha incorrecta 
DataT_Dist.to_csv('Resultados_T/MovilP_T-{}.csv'.format(fecha_str),index=False)

## Creo diferencia vs Semana de REF

In [2]:
path = 'Resultados_T/*'
tipo_de_archivo = '*.csv'
archivos = glob.glob(path + tipo_de_archivo)
# Ordena los archivos por fecha de creación
archivos_ordenados = sorted(archivos, key=os.path.getctime, reverse=True)

#Defino el intervalo par acalcular la diferencia 
Week_Int = 4
nombres_de_archivos = [os.path.basename(archivo)[:-4] for archivo in archivos_ordenados[:1] + archivos_ordenados[Week_Int-1:Week_Int]] 
# Devuelve una lista del archivo más reciente y del segundo más antiguo
nombres_de_archivos


['MovilP_T-31-01-2024', 'MovilP_T-10-01-2024']

In [3]:
dfs = GD.DicofDf(path[:-2])

In [4]:
DataT_Dist = GD.CalcDiffV2(dfs,nombres_de_archivos,['Users','Market Share Claro FB %']) 


La diferencia de Users a nivel del Norte es:  -11443.0
La diferencia de Market Share Claro FB % a nivel del Norte es:  3.999499999999999


In [ ]:
DataT_Dist = DataT_Dist.drop(columns=['Transporte','Capacidad'])

In [8]:
DataT_Dist.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   TD_2G                              397 non-null    float64
 1   TD_3G                              397 non-null    float64
 2   TD_4G                              397 non-null    float64
 3   DL_OSIP_Claro                      351 non-null    float64
 4   DL_OSIP_Movistar                   351 non-null    float64
 5   DL_OSIP_Entel                      351 non-null    float64
 6   DL_OSIP_Bitel                      351 non-null    float64
 7   MayorCompetidor                    397 non-null    object 
 8   MejorRed                           397 non-null    object 
 9   Users_Encendidos                   397 non-null    float64
 10  Users_Voz                          397 non-null    float64
 11  Users_Datos                        397 non-null    float64

In [9]:
DataT_Dist.to_csv('Resultados_Now/Movil_Now.csv',index=False) #DAta actual


# 4 Superset

In [ ]:
chrome_options = Options()
chrome_options.add_extension('Extensione/1.0.0_0.crx')
driver = webdriver.Chrome(options=chrome_options)
############ Me Logeo en el Superset ###################

driver.get("http://172.19.122.127:8087/login/")
assert "Superset" in driver.title
GD.login(driver,'username','password',password='Claro123')

########################################3
driver.get("http://172.19.122.127:8087/superset/dashboard/26/")
time.sleep(3)
assert "Reporte Trafico Semanal" in driver.title, 'Pagina Caida'
time.sleep(5)

#//*[@id="TABS-x8ktpxAlX0-tab-TAB-og-QqVewO"]/div/div/span[1]/span

Sitios = driver.find_element(By.XPATH,('//*[@id="TABS-x8ktpxAlX0-tab-TAB-og-QqVewO"]/div/div/span[1]'))
Sitios.click()
tredoths = driver.find_element(By.XPATH,('//*[@id="slice_919-controls"]/div'))
tredoths.click()
Export = driver.find_element(By.XPATH,('//*[@id="TABS-x8ktpxAlX0-panel-TAB-og-QqVewO"]/div/div/div/div/div/div/div/div[2]/div/div/ul/li[7]'))
Export.click()
time.sleep(3)
driver.close()
# Especifica el path y el tipo de archivo
path = 'C:/Users/C26764/Downloads/*'
tipo_de_archivo = '*.csv'
archivos = glob.glob(path + tipo_de_archivo)
# Busca el archivo más reciente
#archivos
archivo_mas_reciente = max(archivos, key=os.path.getctime)
name_recent = os.path.basename(archivo_mas_reciente)
name_recent


In [ ]:
PathTraf = 'C:/Users/C26764/Downloads/{}'.format(name_recent) #Archivo que aun me falta por descargar del FBSHARE
Df_Traf = pd.read_csv(PathTraf)
Norte = ['AMAZONAS', 'ANCASH', 'CAJAMARCA', 'LA LIBERTAD', 'LAMBAYEQUE', 'TUMBES','PIURA']
Df_Traf_N = Df_Traf[Df_Traf['sub_region'].isin(Norte)]
Df_Traf_N      
Df_Traf_N.to_csv('D:/bi/Data/SitexDistrito.csv',index=False)

In [ ]:
Df_Traf_N